# 네이버 뉴스 크롤링

<img src="http://i.imgur.com/MqZEGQC.jpg" >


#### 2019 FinanceData.KR

## 네이버 뉴스 구성

sid1(대분류)와 sid2(중분류)를 사용하여 뉴스 분야를 지정합니다.

파라미터
* sid1(대분류): 100=정치, 101=경제, 102=사회, 103=생활문화, 104=세계, 105=IT/과학
* sid2(중분류)
  * 정치(sid1=100): 264=청와대, 265=국회/정당, 268=북한, 266=행정, 267=국방/외교, 269=정치일반
  * 경제(sid1=101): 258=증권, 261=산업/재계, 771=중기/벤처, 260=부동산, 262=글로벌 경제, 310=생활경제, 263=경제 일반
  * 사회(sid1=102): 249=사건사고, 250=교육, 251=노동, 254=언론, 252=환경, 59b=인권/복지, 255=식품/의료, 256=지역, 276=인물, 257=사회 일반
  * 생활문화(sid1=103):  241=건강정보, 239=자동차/시승기, 240=도로/교통, 237=여행/레저, 238=음식/맛집, 376=패션/뷰티, 242=공연/전시, 243=책, 244=종교, 248=날씨, 245=생활문화 일반
  * 세계(sid1=104): 231=아시아/호주, 232=미국/중남미, 233=유럽, 234=중동/아프리카, 322=세계 일반
  * IT/과학(sid1=105): 731=모바일, 226=인터넷/SNS, 227=통신/뉴미디어, 230=IT 일반, 732=보안/해킹, 283=컴퓨터, 229=게임/리뷰, 228=과학 일반
* date=20180110 날짜를 지정해서 특정 날짜의 목록

예를 들어, "경제 / 경제 일반" 영역의 경우 URL은 다음과 같습니다

https://news.naver.com/main/list.nhn?sid1=101&sid2=263&date=20190104&page=1


<img src="https://i.imgur.com/aKfkdhE.png">



# 필요 모듈 가져오기및 설정

In [0]:
!pip install -q newspaper3k

In [0]:
import requests
from bs4 import BeautifulSoup

import dateutil
import io
import numpy as np
import pandas as pd

# 지정 영역 뉴스 목록 가져오기

"경제/경제 일반" (sid1=101, sid2=263) 영역의 뉴스 목록 가져오기

In [0]:
# "경제/경제 일반" 페이지의 뉴스 목록 가져오기
import requests
from bs4 import BeautifulSoup

url_tmpl = 'https://news.naver.com/main/list.nhn' \
           '?mode=LS2D&mid=shm&sid1=101&sid2=263&date=%s&page=%s'

date, page = '20190104', '1'
r = requests.get(url_tmpl % (date, page))
soup = BeautifulSoup(r.text, 'html.parser')
main_content = soup.find(id='main_content')
li_list = main_content.find_all('li')
len(li_list)

20

In [0]:
li_list[0]

<li>
<dl>
<dt>
<a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=263&amp;oid=009&amp;aid=0004283358">
								 EU, 韓철강에 세이프가드…내달부터 초과물량 25% 관세
								</a>
</dt>
<dd>
<span class="lede">유럽연합(EU)이 다음달 2일부터 일정 수입 물량을 초과하는 한국산 철강에 25% 관세를 물리는 세이프가드(긴급수입제한조치)를  …</span>
<span class="writing">매일경제</span>
<span class="newspaper_info" title="신문에 게재되었으며 A8면의 기사입니다."><i class="picon">신문</i>A8면 </span>
<span class="date">2019.01.04. 오후 11:27</span>
</dd>
</dl>
</li>

In [0]:
li_list[1]

<li>
<dl>
<dt class="photo">
<a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=263&amp;oid=001&amp;aid=0010563067">
<img alt="美 작년 12월 일자리 큰 폭 증가…실업률 3.9%로 상승" height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.news/image/news/2009/noimage_106x72.png';" src="https://imgnews.pstatic.net/image/origin/001/2019/01/04/10563067.jpg?type=nf106_72" width="106">
</img></a>
</dt>
<dt>
<a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=263&amp;oid=001&amp;aid=0010563067">
								 美 작년 12월 일자리 큰 폭 증가…실업률 3.9%로 상승
								</a>
</dt>
<dd>
<span class="lede">(뉴욕=연합뉴스) 이귀원 특파원 = 경기둔화 우려 속에서도 미국의 지난달 일자리가 큰 폭으로 증가했다. 신규 노동인구 유입으로  …</span>
<span class="writing">연합뉴스</span>
<span class="date">2019.01.04. 오후 11:17</span>
</dd>
</dl>
</li>

In [0]:
# 날짜 파싱하기
import dateutil

date_str = '2019.01.04. 오후 11:17'
date_str = date_str.replace('오후', '') + ' PM' if '오후' in date_str else date_str
date_str = date_str.replace('오전', '') + ' AM' if '오전' in date_str else date_str

dateutil.parser.parse(date_str)

datetime.datetime(2019, 1, 4, 23, 17)

In [0]:
# 전체 뉴스 목록

for li in li_list:
    # 뉴스 개괄(제목, 링크, 서두, 날짜)
    dt = li.find('dt', attrs=None)
    link = dt.a['href']
    title = dt.text.strip()
    lede = li.find('span', attrs='lede').text
    writing = li.find('span', attrs='writing').text
    date_str = li.find('span', attrs='date').text
    date_str = date_str.replace('오후', '') + ' PM' if '오후' in date_str else date_str
    date_str = date_str.replace('오전', '') + ' AM' if '오전' in date_str else date_str
    date = dateutil.parser.parse(date_str)

    print(date, title)
    print(link)


2019-01-04 23:27:00 EU, 韓철강에 세이프가드…내달부터 초과물량 25% 관세
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&oid=009&aid=0004283358
2019-01-04 23:17:00 美 작년 12월 일자리 큰 폭 증가…실업률 3.9%로 상승
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&oid=001&aid=0010563067
2019-01-04 23:14:00 美, 12월 일자리 31만2000개 늘어…실업률 3.9%
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&oid=003&aid=0008995366
2019-01-04 22:46:00 EU, 다음달 2일부터 철강 세이프가드 시행…최종조치 계획 WTO 통보
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&oid=277&aid=0004388553
2019-01-04 22:33:00 국토부 "토지 공시가 100% 인상 지침 없다"
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&oid=422&aid=0000355084
2019-01-04 22:32:00 새해도 인기지역 청약 열기 '활활'…나머지는 '꽁꽁'
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&oid=422&aid=0000355083
2019-01-04 22:25:00 "파업으로 영업차질시 책임" 국민은 경영진 사직서
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=10

## 개별 뉴스 1건 읽어오기

다음과 같은 뉴스 링크

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&sid2=263&oid=001&aid=0010563076


<img src="https://i.imgur.com/r7tZIYD.png" >

## 네이버 뉴스 페이지에서 텍스트 추출

스크립트(script) 부분은 필요하지 않으므로 script 부분을 삭제

In [0]:
url='https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&sid2=263&oid=001&aid=0010563076'

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
soup.title.text

'뉴욕증시, 美고용 깜짝 호조·中 부양책 상승 출발 : 네이버 뉴스'

In [0]:
soup.find(id='articleBodyContents')

<div id="articleBodyContents">
<!-- 본문 내용 -->
<!-- TV플레이어 -->
<!-- // TV플레이어 -->
<script type="text/javascript">
// flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}
</script>
	
	(뉴욕=연합뉴스) 오진우 연합인포맥스 특파원 = 뉴욕증시에서 주요 지수는 4일 미국 고용지표가 큰 폭 호조를 보인 데다, 중국도 경기 부양책을 내놓은 데 힘입어 상승 출발했다.<br/><br/>    오전 9시 39분(미 동부시간) 현재 뉴욕증권거래소(NYSE)에서 다우존스 30 산업평균지수는 전장보다 355.48포인트(1.57%) 상승한 23,041.70에 거래됐다.<br/><br/>    스탠더드앤드푸어스(S&amp;P) 500 지수는 전장보다 39.77포인트(1.62%) 오른 2,487.66에, 기술주 중심의 나스닥 지수는 115.88포인트(1.79%) 상승한 6,579.38에 거래됐다.<br/><br/>    시장은 중국의 지급준비율(지준율·RRR) 인하 등 부양책과 무역협상 관련 소식, 미국 고용지표 및 제롬 파월 연방준비제도(Fed·연준) 의장의 발언 등을 주시했다.<br/><br/>    미 노동부는 12월 비농업 신규고용이 31만2천 명 증가했다고 밝혔다. 월스트리트저널(WSJ)이 집계한 시장 예상치 17만6천 명을 훌쩍 뛰어넘었다.<br/><br/>    지난달 실업률은 3.9%로 전달 3.7%에서 올랐지만, 이는 노동시장 참가율이 지난 2014년 이후 최고인 63.1%로 오른 영향으로 풀이됐다.<br/><br/>    또 시간당 임금은 지난해 같은 달보다 3.2% 올라 2008년 이후 최고치를 기록하는 등 고용지표가 전방위적으로 강했다.<br/><br/>    WSJ은 강한 고용지표가 최근 점증하는 미국 경기 둔화 우려에 대한 균형추가 될 수 있다고 진단했다.<br/><br/>    고용지표 발표 이후 미 국채

In [0]:
article = soup.find(id='articleBodyContents')
article

<div id="articleBodyContents">
<!-- 본문 내용 -->
<!-- TV플레이어 -->
<!-- // TV플레이어 -->
<script type="text/javascript">
// flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}
</script>
	
	(뉴욕=연합뉴스) 오진우 연합인포맥스 특파원 = 뉴욕증시에서 주요 지수는 4일 미국 고용지표가 큰 폭 호조를 보인 데다, 중국도 경기 부양책을 내놓은 데 힘입어 상승 출발했다.<br/><br/>    오전 9시 39분(미 동부시간) 현재 뉴욕증권거래소(NYSE)에서 다우존스 30 산업평균지수는 전장보다 355.48포인트(1.57%) 상승한 23,041.70에 거래됐다.<br/><br/>    스탠더드앤드푸어스(S&amp;P) 500 지수는 전장보다 39.77포인트(1.62%) 오른 2,487.66에, 기술주 중심의 나스닥 지수는 115.88포인트(1.79%) 상승한 6,579.38에 거래됐다.<br/><br/>    시장은 중국의 지급준비율(지준율·RRR) 인하 등 부양책과 무역협상 관련 소식, 미국 고용지표 및 제롬 파월 연방준비제도(Fed·연준) 의장의 발언 등을 주시했다.<br/><br/>    미 노동부는 12월 비농업 신규고용이 31만2천 명 증가했다고 밝혔다. 월스트리트저널(WSJ)이 집계한 시장 예상치 17만6천 명을 훌쩍 뛰어넘었다.<br/><br/>    지난달 실업률은 3.9%로 전달 3.7%에서 올랐지만, 이는 노동시장 참가율이 지난 2014년 이후 최고인 63.1%로 오른 영향으로 풀이됐다.<br/><br/>    또 시간당 임금은 지난해 같은 달보다 3.2% 올라 2008년 이후 최고치를 기록하는 등 고용지표가 전방위적으로 강했다.<br/><br/>    WSJ은 강한 고용지표가 최근 점증하는 미국 경기 둔화 우려에 대한 균형추가 될 수 있다고 진단했다.<br/><br/>    고용지표 발표 이후 미 국채

In [0]:
article.script.extract()

<script type="text/javascript">
// flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}
</script>

In [0]:
article.text.strip()

'(뉴욕=연합뉴스) 오진우 연합인포맥스 특파원 = 뉴욕증시에서 주요 지수는 4일 미국 고용지표가 큰 폭 호조를 보인 데다, 중국도 경기 부양책을 내놓은 데 힘입어 상승 출발했다.    오전 9시 39분(미 동부시간) 현재 뉴욕증권거래소(NYSE)에서 다우존스 30 산업평균지수는 전장보다 355.48포인트(1.57%) 상승한 23,041.70에 거래됐다.    스탠더드앤드푸어스(S&P) 500 지수는 전장보다 39.77포인트(1.62%) 오른 2,487.66에, 기술주 중심의 나스닥 지수는 115.88포인트(1.79%) 상승한 6,579.38에 거래됐다.    시장은 중국의 지급준비율(지준율·RRR) 인하 등 부양책과 무역협상 관련 소식, 미국 고용지표 및 제롬 파월 연방준비제도(Fed·연준) 의장의 발언 등을 주시했다.    미 노동부는 12월 비농업 신규고용이 31만2천 명 증가했다고 밝혔다. 월스트리트저널(WSJ)이 집계한 시장 예상치 17만6천 명을 훌쩍 뛰어넘었다.    지난달 실업률은 3.9%로 전달 3.7%에서 올랐지만, 이는 노동시장 참가율이 지난 2014년 이후 최고인 63.1%로 오른 영향으로 풀이됐다.    또 시간당 임금은 지난해 같은 달보다 3.2% 올라 2008년 이후 최고치를 기록하는 등 고용지표가 전방위적으로 강했다.    WSJ은 강한 고용지표가 최근 점증하는 미국 경기 둔화 우려에 대한 균형추가 될 수 있다고 진단했다.    고용지표 발표 이후 미 국채금리가 급등하고 달러도 강세를 보이는 등 금융시장의 위험회피 심리가 완화했다.    중국에서도 긍정적인 소식이 나왔다.    중국 중앙은행인 인민은행은 이날 지준율을 100bp(1%포인트) 인하한다고 밝혔다.    최근 각종 경제지표가 눈에 띄게 악화하는 가운데 나온 조치로, 당국의 경기 방어 의지가 반영된 것으로 풀이된다.    중국은 지난해에도 지준율을 네 차례 인하했다.    중국 상무부는 또 오는 7일과 8일 미국 실무 협상단이 중국을 방문해 무역 협상을 진행한다는 점을 재차

## 함수로 정리 - 개별 뉴스 1건의 제목과 본문 가져오기 

In [0]:
## 함수로 정리

def get_naver_news_text(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 제목 가져오기
    title = soup.title.text
    # 본문 가져오기
    article = soup.find(id='articleBodyContents')
    article.script.extract()
    text = article.text.strip()
    return title, text

In [0]:
## 함수 테스트

url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=366&aid=0000431718'
title, text = get_naver_news_text(url)

print(title)
print('-' * 100)
print(text)

또 1兆 적자 낸 쿠팡, 치킨게임 승자되나...누적적자 2.8兆 : 네이버 뉴스
----------------------------------------------------------------------------------------------------
쿠팡 지난해 1조1190억원 적자...누적적자 2.8조로켓배송·로켓프레시...물류비 1000억원 늘어 매출 4.4조...위메프·티몬의 10배 국내 소셜커머스업체 쿠팡이 지난해 1조1190억원(개별 재무제표 기준)의 당기순손실을 기록했다. 이는 전년(6570억원)보다 적자폭이 167% 가량 확대된 것이다. 이로써 최근 4년간 누적적자는 2조8640억원으로 늘었다. 이는 손정의 소프트뱅크 회장이 최초 투자한 1조1000억원에 더해 지난해 11월 추가 투자한 2조3000억원을 합한 총 투자금액(3조4000억원)의 84%에 달하는 금액이다. 쿠팡 로켓배송/조선DB15일 쿠팡이 금융감독원에 제출한 감사보고서에 따르면 쿠팡의 작년말 기준 자본총계는 117억6300만원이다. 전년 마이너스(-) 2446억원으로 완전 자본잠식 상태였으나 손 회장이 이끄는 비전펀드에서 추가 투자를 받아 자본잠식을 탈피했다. 완전 자본잠식은 회사의 적자가 계속돼 납입자본금마저 바닥이 난 상태를 말한다. 쿠팡이 1조원이 넘는 적자를 기록하면서 결손금도 크게 늘었다. 작년말 기준 결손금은 2조9849억원이다. 전년(1조8660억원)보다 대폭 확대됐다. 결손금이 늘었다는 것은 자본이 그만큼 줄었다는 것을 의미한다. 쿠팡의 적자폭이 확대된 이유는 크게 세 가지다. 쿠팡 직매입 상품을 하루만에 배송하는 자체 배송망 ‘로켓배송’, 여기다 신석식품 새벽배송에 따른 물류비 증가와 인건비·이자비용 증가다. 로켓배송은 당일 밤 12시까지 주문하면 다음 날 바로 집 앞 현관으로 배송되는 시스템이다. 쿠팡은 로켓배송을 위해 직매입 시스템을 유지하고 있다. 전국 지역 물류센터를 기존 12개에서 24개로 늘렸다. 또 지난해 10월 신선식품 새벽배송 서비스인 로켓프

## 파일로 저장, 확인, 다운로드(Colab)

In [0]:
# 파일로 저장하기

url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=366&aid=0000431718'
title, text = get_naver_news_text(url)

with open('sample_news.txt', 'w') as f:
    f.write(text)

In [0]:
# 파일 확인
!ls -lh sample_news.txt

-rw-r--r-- 1 root root 4.3K Jun  1 00:46 sample_news.txt


In [0]:
!cat sample_news.txt

쿠팡 지난해 1조1190억원 적자...누적적자 2.8조로켓배송·로켓프레시...물류비 1000억원 늘어 매출 4.4조...위메프·티몬의 10배 국내 소셜커머스업체 쿠팡이 지난해 1조1190억원(개별 재무제표 기준)의 당기순손실을 기록했다. 이는 전년(6570억원)보다 적자폭이 167% 가량 확대된 것이다. 이로써 최근 4년간 누적적자는 2조8640억원으로 늘었다. 이는 손정의 소프트뱅크 회장이 최초 투자한 1조1000억원에 더해 지난해 11월 추가 투자한 2조3000억원을 합한 총 투자금액(3조4000억원)의 84%에 달하는 금액이다. 쿠팡 로켓배송/조선DB15일 쿠팡이 금융감독원에 제출한 감사보고서에 따르면 쿠팡의 작년말 기준 자본총계는 117억6300만원이다. 전년 마이너스(-) 2446억원으로 완전 자본잠식 상태였으나 손 회장이 이끄는 비전펀드에서 추가 투자를 받아 자본잠식을 탈피했다. 완전 자본잠식은 회사의 적자가 계속돼 납입자본금마저 바닥이 난 상태를 말한다. 쿠팡이 1조원이 넘는 적자를 기록하면서 결손금도 크게 늘었다. 작년말 기준 결손금은 2조9849억원이다. 전년(1조8660억원)보다 대폭 확대됐다. 결손금이 늘었다는 것은 자본이 그만큼 줄었다는 것을 의미한다. 쿠팡의 적자폭이 확대된 이유는 크게 세 가지다. 쿠팡 직매입 상품을 하루만에 배송하는 자체 배송망 ‘로켓배송’, 여기다 신석식품 새벽배송에 따른 물류비 증가와 인건비·이자비용 증가다. 로켓배송은 당일 밤 12시까지 주문하면 다음 날 바로 집 앞 현관으로 배송되는 시스템이다. 쿠팡은 로켓배송을 위해 직매입 시스템을 유지하고 있다. 전국 지역 물류센터를 기존 12개에서 24개로 늘렸다. 또 지난해 10월 신선식품 새벽배송 서비스인 로켓프레시를 시작했다. 고객이 자정까지 신선식품을 주문하면 오전 7시 전에 배송해 준다. 이런 규모의 상품을 고객에게 익일 배송할 수 있는 유통사는 쿠팡이 유일하다.김범석 쿠팡 대표는 "우리는 고객을 감동시키기 위해 어느 누구도 시도하지 않았던 막대한 투자를 진행해 

In [0]:
# (구글 Colab 경우) 로컬 PC로 다운로드 하기
from google.colab import files

files.download('sample_news.txt')

# 재사용 가능한 함수로 정리

## 특정 날짜의 지정 영역 뉴스 링크 모두 가져오기

"경제/경제 일반" 
뉴스 본문을 식별하여 가져오기 위해  newspaper ( https://newspaper.readthedocs.io ) 활용

In [0]:
from datetime import datetime
import pandas as pd
import newspaper
import requests
from bs4 import BeautifulSoup

def news_naver(date=None, sid1='101', sid2='263', verbose=False):
  date = datetime.today() if date==None else pd.to_datetime(date)

  url_tmpl = 'https://news.naver.com/main/list.nhn' \
             '?mode=LS2D&mid=shm&sid1=%s&sid2=%s&date=%s&page=%s'

  page = 0
  prev_content_text = ''
  row_list = []
  while True:
    page += 1
    url = url_tmpl % (sid1, sid2, date.strftime('%Y%m%d'), str(page))
    if verbose:
      print(url, end=' ')
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    main_content = soup.find(id='main_content')
    if prev_content_text == main_content.text:
      break
    prev_content_text = main_content.text
    li_list = main_content.find_all('li')

    for li in li_list:
      # 뉴스 개괄(제목, 링크, 서두, 날짜)
      dt_tag = li.find('dt', attrs=None)
      link = dt_tag.a['href']
      title = dt_tag.text.strip()
      lede = li.find('span', attrs='lede').text
      writing_tag = li.find('span', attrs='writing')
      writing = writing_tag.text if writing_tag else ''
      date_str = li.find('span', attrs='date').text
      date_str = date_str.replace('오후', '') + ' PM' if '오후' in date_str else date_str
      date_str = date_str.replace('오전', '') + ' AM' if '오전' in date_str else date_str
      date = dateutil.parser.parse(date_str)

      # 뉴스 본문
      article = newspaper.Article(link)
      article.download()
      article.parse()
      text = article.text
      row_list.append([date, title, link, lede, writing, text])
    if verbose:
      print(' %d cum_rows ' % len(row_list))
  cols = ['Date', 'Title', 'Link', 'Lede', 'Writing', 'Text']
  df = pd.DataFrame(row_list, columns=cols)
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
  return df

In [0]:
news_naver('2019-03-01', sid1='101', sid2='263', verbose=True)

https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&date=20190301&page=1  20 cum_rows 
https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&date=20190301&page=2  40 cum_rows 
https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&date=20190301&page=3  60 cum_rows 
https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&date=20190301&page=4  80 cum_rows 
https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&date=20190301&page=5  100 cum_rows 
https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&date=20190301&page=6  120 cum_rows 
https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&date=20190301&page=7  140 cum_rows 
https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&date=20190301&page=8  160 cum_rows 
https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&date=20190301&page=9  180 cum_rows 
https://news.naver.com/

,Date,Title,Link,Lede,Writing,Text
0,2019-03-01 23:53:00,[고침] 경제('반도체·중국 부진'에 2월 수출 11.7%↓…석 달 연속 감소(종합)),https://news.naver.com/main/read.nhn?mode=LS2D...,'반도체·중국 부진'에 2월 수출 11.1%↓…석 달 연속 감소(종합) 반도체 수출...,연합뉴스,2월 수출 11.1%↓…석달 연속 감소(CG)[연합뉴스TV 제공] 2월 수출 11....
1,2019-03-01 23:44:00,美 12월 소비지출 0.5%↓…9년래 최대폭 감소,https://news.naver.com/main/read.nhn?mode=LS2D...,【로스앤젤레스=뉴시스】 류강훈 기자 = 미국의 지난해 12월 소비자 지출이 0.5%...,뉴시스,【스크랜턴(미 펜실베이니아주)=AP/뉴시스】 미국의 지난해 12월 소비자 지출이 0...
2,2019-03-01 23:31:00,"철도·도로 연결 '일단 멈춤'...""협의는 지속""",https://news.naver.com/main/read.nhn?mode=LS2D...,"[앵커] 2차 북미 정상회담이 합의 없이 끝나면서 남북 경제협력 사업, 특히 철도와...",YTN,동영상 뉴스\n\n[앵커]2차 북미 정상회담이 합의 없이 끝나면서 남북 경제협력 사...
3,2019-03-01 23:28:00,'北 해제 요구 5건'...韓 광물·수산물 수입 영향있나?,https://news.naver.com/main/read.nhn?mode=LS2D...,[앵커] 과거 남북 관계가 좋을 때 경협사업 외에도 수산물과 광물 관련 교역도 활발...,YTN,동영상 뉴스\n\n[앵커]과거 남북 관계가 좋을 때 경협사업 외에도 수산물과 광물 ...
4,2019-03-01 23:12:00,"대기업 신입 공채 개막...""다음 달 결론""",https://news.naver.com/main/read.nhn?mode=LS2D...,주요 대기업이 이번 달부터 올해 상반기 신입사원 공채에 본격적으로 나섭니다. 현대차...,YTN,동영상 뉴스\n\n주요 대기업이 이번 달부터 올해 상반기 신입사원 공채에 본격적으로...
5,2019-03-01 23:11:00,8일부터 '동탄→수서' SRT 출근 열차 운행,https://news.naver.com/main/read.nhn?mode=LS2D...,"오는 8일부터 매일 아침 동탄역에서 수서역까지 운행하는 고속철도, SRT 출근열차가...",YTN,"동영상 뉴스\n\n오는 8일부터 매일 아침 동탄역에서 수서역까지 운행하는 고속철도,..."
6,2019-03-01 23:08:00,불법 전매 수익 천만 원 넘으면 3배 벌금,https://news.naver.com/main/read.nhn?mode=LS2D...,분양권의 불법 전매를 하거나 알선하다 걸리면 수익의 3배까지 벌금을 물게 됩니다. ...,YTN,동영상 뉴스\n\n분양권의 불법 전매를 하거나 알선하다 걸리면 수익의 3배까지 벌금...
7,2019-03-01 23:06:00,"2월 수출 11% 감소...""반도체·중국 부진""",https://news.naver.com/main/read.nhn?mode=LS2D...,[앵커] 지난달 수출이 큰 폭으로 줄어든 것으로 나타났습니다. 정부는 올해 설 연휴...,YTN,동영상 뉴스\n\n[앵커]지난달 수출이 큰 폭으로 줄어든 것으로 나타났습니다.정부는...
8,2019-03-01 23:05:00,영국 2월 제조업 PMI 52.0…4개월래 최저치,https://news.naver.com/main/read.nhn?mode=LS2D...,【서울=뉴시스】권성근 기자 = 브렉시트(영국 유럽연합(EU) 탈퇴)에 따른 불확실성...,뉴시스,【서울=뉴시스】권성근 기자 = 브렉시트(영국 유럽연합(EU) 탈퇴)에 따른 불확실성...
9,2019-03-01 22:37:00,英 의회 직원 임금인상률 1.5%·의원은 2.7%…노동계 반발,https://news.naver.com/main/read.nhn?mode=LS2D...,(런던=연합뉴스) 박대한 특파원 = 영국 하원의원들이 물가는 물론 하원에서 일하는 ...,연합뉴스,영국 하원의 모습 [EPA=연합뉴스] 영국 하원의 모습 [EPA=연합뉴스]\n\n(...


#### 2019 FinanceData.KR